## Load libraries

In [4]:
import json
import pandas as pd
import os
import sys
import sklearn
import datetime
import numpy as np
import requests


# turn off warnings
import warnings
warnings.filterwarnings('ignore')


# set all columns to be displayed
pd.set_option('display.max_columns', None)

from dotenv import load_dotenv
load_dotenv()



from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import time

## Load data

In [2]:
data_test = pd.read_excel('data/Test_check.xlsx', sheet_name='sasha_3')
data_test.head()

,URL,lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп..."


## Selenium initialize

Start docker container with silenium before run code.

In [61]:
def get_chrome_driver(hub_url):
    """Initialize and return a new Chrome WebDriver instance."""
    chrome_options = ChromeOptions()
    chrome_options.page_load_strategy = 'normal'
    
    try:
        driver = webdriver.Remote(command_executor=hub_url, options=chrome_options)
        return driver
    except WebDriverException as e:
        print(f"Error initializing WebDriver: {e}")
        return None

def check_and_renew_driver(driver: webdriver, hub_url: str) -> webdriver:
    """Check if the driver is alive and renew if necessary."""
    try:
        driver.title  # Attempting to access a property to check if it's still active
        return driver  # Driver is still active
    except (WebDriverException, AttributeError):
        print("WebDriver is not active. Reinitializing...")
        return get_chrome_driver(hub_url)
    
hub_url = "http://localhost:4444/wd/hub" # docker container with selenium and chrome
# hub_url = "http://chrome:4444/wd/hub" # docker container with spark and chrome. chrome is a service name in docker-compose.yml

# Example usage
# chrome_driver = get_chrome_driver(hub_url)

# # Later in the code, before using the driver:
# chrome_driver = check_and_renew_driver(chrome_driver, hub_url)

def close_driver(driver):
    """Close the WebDriver instance."""
    try:
        driver.quit()
    except WebDriverException as e:
        print(f"Error closing WebDriver: {e}")

## Get news by URL

In [8]:
chrome_driver = get_chrome_driver(hub_url)
# close_driver(chrome_driver)

In [9]:
# chrome_driver = get_chrome_driver(hub_url)
chrome_driver = check_and_renew_driver(chrome_driver, hub_url)

In [3]:
url_list = data_test['URL'].tolist()
url_list

['https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/',
 'https://ria.ru/20250311/klyuchevaya_stavka-1962773695.html',
 'https://expert.ru/mnenie/denis-manturov-gosudarstvo-v-ramkakh-natsproektov-obespechivaet-preferentsii-rossiyskim-predpriyatiya/',
 'https://ria.ru/20250304/kredity-2003042476.html',
 'https://lenta.ru/brief/2025/03/11/green/',
 'https://lenta.ru/articles/2025/03/09/aslan/',
 'https://expert.ru/finance/investorov-svyazyvayut-blokcheynom/',
 'https://ria.ru/20250123/svo-1985822676.html',
 'https://expert.ru/mnenie/vse-teper-zavisit-tolko-ot-voli-gosudarstva/',
 'https://ria.ru/docs/about/privacy_policy.html',
 'https://ria.ru/docs/about/privacy_policy.html#1748006692-13',
 'https://lenta.ru/extlink/2025/03/10/na-ukraine-ispugalis-skorogo-otklyucheniya-starlink/?intlnk=true',
 'https://lenta.ru/news/2025/03/10/na-ukraine-ispugalis-skorogo-otklyucheniya-starlink-pochemu-vsu-tak-vazhen-sputnikovyy-internet-ot-maska/',
 'https://expert.ru/news/rynok-onlayn-

In [ ]:
# with selenium
# def get_text_from_expert(url, driver, timeout=10):
#     """Fetch the text content from a news page inside <div class='section-main-box'>."""
#     try:
#         driver.get(url)
        
#         # Wait until the specific div is present
#         WebDriverWait(driver, timeout).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "section-main-box"))
#         )

#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         main_section = soup.find("div", class_="section-main-box")
        
#         if not main_section:
#             print(f"[Warning] No div with class 'section-main-box' found in {url}")
#             return None

#         paragraphs = main_section.find_all('p')
#         text_content = ' '.join(para.get_text(strip=True) for para in paragraphs)

#         return text_content if text_content else None

#     except Exception as e:
#         print(f"[Error] Failed to fetch text from {url}: {e}")
#         return None


In [54]:
def get_text_from_expert(url, timeout=10):

    try:
        article_text = []

        response = requests.get(url, timeout=timeout)
        response.raise_for_status()  # Raise an error for bad responses

        soup = BeautifulSoup(response.text, 'html.parser')

        # Headline
        article_text.append(soup.find("h1", itemprop="headline").get_text(strip=True))

        # Subtitle
        article_text.append(soup.find("h2", class_="subtitle_article").get_text(strip=True))

        # Theme
        article_text.append(soup.select_one(".article-theme").get_text(strip=True))

        # Date
        article_text.append(soup.find("span", class_="article-date")["content"].split('T')[0])

        # Author
        article_text.append(soup.select_one(".article-header-author-bl [itemprop='name']").get_text(strip=True))

        # Article body
        article_body = soup.find("div", class_="plain-text")
        paragraphs = [p.get_text(strip=True) for p in article_body.find_all("p")]
        main_text = "\n\n".join(paragraphs)
        article_text.append(main_text)
        # Join all parts into a single string
        article_text = "\n".join(article_text)
        # Clean up the text from \xa0 and extra spaces
        article_text = re.sub(r'\s+', ' ', article_text)  # Replace multiple spaces with a single space
        article_text = article_text.replace('\xa0', '').strip()




        return article_text

    except Exception as e:
        print(f"[Error] Failed to parse article from {url}: {e}")
        return None

In [56]:
def get_text_from_ria(url, timeout=10):
    """Fetch the text content from a 'ria' news page using BeautifulSoup."""
    try:
        response = requests.get(url, timeout=timeout)
        soup = BeautifulSoup(response.content, 'html.parser')

        article_text = []

        # 1. Ищем все блоки с классом "article__block"
        for block in soup.find_all('div', class_='article__block'):
            block_type = block.get('data-type')

            # Заголовки (например, h2)
            if block_type == 'h2':
                header = block.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
                if header:
                    article_text.append(header.get_text(strip=True))

            # Обычные текстовые блоки
            elif block_type == 'text':
                text_block = block.find('div', class_='article__text')
                if text_block:
                    article_text.append(text_block.get_text(strip=True))

            # Списки дат или других элементов
            elif block_type == 'list':
                for ul in block.find_all('ul', class_='article__list m-circle'):
                    for li in ul.find_all('li', class_='article__list-item'):
                        full_text = li.get_text(strip=True)
                        if full_text:
                            article_text.append(f"  • {full_text}")

        # 2. Альтернативный вариант: если текст не найден, пробуем "article__body"
        if not article_text:
            article_body = soup.find('div', class_='article__body')
            if article_body:
                for p in article_body.find_all('p'):
                    article_text.append(p.get_text(strip=True))

        # Объединяем весь текст с переносами строк
        full_text = '\n'.join(article_text)

        return full_text
    
    except Exception as e:
        print(f"[Error] Failed to fetch text from {url}: {e}")
        return None


In [ ]:
def get_text_from_lenta_brief(url):
    """Fetch the brief content from a 'lenta.ru' news page using BeautifulSoup."""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        article_text = []

        # Extract publication date
        article_text.append(soup.select_one('a.topic-header__time'))
       
        # Extract category
        article_text.append(soup.select_one('a.topic-header__rubric'))
     
        # Extract full title
        title_parts = soup.select('h1.topic-body__titles span')
        article_text.append(" ".join([part.get_text(strip=True) for part in title_parts]))

        # Extract article sections
        brief_cards = soup.select('div.box-brief-card')

        for card in brief_cards:
            # Extracting the number, subtitle, and content from each card
            article_text.append(card.select_one('span.box-brief-card__number').text.strip())
            article_text.append(card.select_one('div.box-brief-card__title').text.strip())
            paragraphs = card.select('p.box-brief-card__content-text')
            article_text.append("\n".join(p.get_text(strip=True) for p in paragraphs))
            
        # Join all parts into a single string
        article_text = "\n".join(article_text)
        
        return article_text.strip() if article_text else None

    except Exception as e:
        print(f"[Error] Failed to fetch text from {url}: {e}")
        return None

In [ ]:
# without selenium doesn't load all content
# def get_text_from_lenta_article(url):
#     """Fetch and parse the article content from a Lenta.ru page using BeautifulSoup."""
#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, 'html.parser')

#         article_text = []

#         # Extract date
#         date_tag = soup.select_one('a.topic-header__time')
#         if date_tag:
#             article_text.append(f"Date: {date_tag.get_text(strip=True)}")

#         # Extract category
#         category_tag = soup.select_one('a.topic-header__rubric')
#         if category_tag:
#             article_text.append(f"Category: {category_tag.get_text(strip=True)}")

#         # Extract title
#         title_parts = soup.select('h1.topic-body__titles span')
#         if title_parts:
#             title = " ".join(part.get_text(strip=True) for part in title_parts)
#             article_text.append(f"Title: {title}")

#         # Extract author
#         author_tag = soup.select_one('a.topic-authors__author span.topic-authors__name')
#         if author_tag:
#             author_name = author_tag.get_text(strip=True)
#             article_text.append(f"Author: {author_name}")

#         # Extract author job/role
#         job_tag = soup.select_one('a.topic-authors__author span.topic-authors__job')
#         if job_tag:
#             article_text.append(f"{job_tag.get_text(strip=True)}")

#         # Extract "lead" and main body content
#         lead_paragraphs = soup.select('p.topic-body__content-text._lead')
#         body_paragraphs = soup.select('p.topic-body__content-text:not(._lead)')

#         if lead_paragraphs or body_paragraphs:
#             article_text.append("\n=== Article Content ===\n")
#             for p in lead_paragraphs + body_paragraphs:
#                 article_text.append(p.get_text(strip=True))

#         # Extract any note blocks (like box-note)
#         note_blocks = soup.select('div.box-note p.box-note__text')
#         for note in note_blocks:
#             article_text.append(f"[NOTE] {note.get_text(strip=True)}")

#         # Optional: Also handle the brief-card structure if present
#         brief_cards = soup.select('div.box-brief-card')
#         for card in brief_cards:
#             number = card.select_one('span.box-brief-card__number')
#             title = card.select_one('div.box-brief-card__title')
#             paragraphs = card.select('p.box-brief-card__content-text')

#             if number or title or paragraphs:
#                 article_text.append("\n---")
#                 if number:
#                     article_text.append(f"[{number.get_text(strip=True)}]")
#                 if title:
#                     article_text.append(title.get_text(strip=True))
#                 if paragraphs:
#                     article_text.extend(p.get_text(strip=True) for p in paragraphs)

#         return "\n".join(article_text).strip() if article_text else None

#     except Exception as e:
#         print(f"[Error] Failed to fetch text from {url}: {e}")
#         return None


In [109]:
def get_text_from_lenta_article(url, driver):
    """Fetch full article text from lenta.ru using Selenium and BeautifulSoup. Scroll until article block is loaded."""
    options = ChromeOptions()
    options.headless = True
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    try:
        driver.get(url)

        # Scroll until <div class="topic-page__wrap js-topic"> is present
        timeout = 15
        end_time = time.time() + timeout
        while time.time() < end_time:
            if driver.find_elements(By.CSS_SELECTOR, "div.topic-page__wrap.js-topic"):
                break
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.5)
        else:
            raise TimeoutError("Failed to load article wrapper div")

        # Wait for article title to ensure full content
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h1.topic-body__titles"))
        )

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        article_text = []

        # Date
        if (tag := soup.select_one('a.topic-header__time')):
            article_text.append(f"Date: {tag.get_text(strip=True)}")

        # Category
        if (tag := soup.select_one('a.topic-header__rubric')):
            article_text.append(f"Category: {tag.get_text(strip=True)}")

        # Title
        title_parts = soup.select('h1.topic-body__titles span')
        if title_parts:
            article_text.append("Title: " + " ".join(part.get_text(strip=True) for part in title_parts))

        # Author and role
        if (tag := soup.select_one('span.topic-authors__name')):
            article_text.append(f"Author: {tag.get_text(strip=True)}")
        if (tag := soup.select_one('span.topic-authors__job')):
            article_text.append(tag.get_text(strip=True))

        article_text.append("\n=== Article Content ===\n")

               # Select all relevant blocks: h2 headings and paragraphs
        content_blocks = soup.select('h2.topic-body__content-title')

        for block in content_blocks:
            # Treat the h2 heading "-"
            heading_text = block.get_text(strip=True).upper()
            article_text.append(f"{heading_text}")
            for paragraph in block.find_next_siblings(['p', 'div']):
                if paragraph.name == 'p':
                    text = paragraph.get_text(strip=True)
                    # Prepend dash if in note_mode or paragraph fully italic (optional)
                    is_note = any(tag.name == 'i' for tag in paragraph.find_all(recursive=False))
                    article_text.append(f"- {text}" if is_note else text)
                elif paragraph.name == 'div' and 'box-note' in paragraph.get('class', []):
                    note_text = paragraph.get_text(strip=True)
                    article_text.append(f"- {note_text}")
        
        return "\n".join(article_text).strip()

    except Exception as e:
        print(f"[Error] Failed to fetch text from {url}: {e}")
        return None


In [ ]:
chrome_driver = get_chrome_driver(hub_url)
# close_driver(chrome_driver)

In [110]:
# chrome_driver = get_chrome_driver(hub_url)
chrome_driver = check_and_renew_driver(chrome_driver, hub_url)

In [111]:
url = 'https://lenta.ru/articles/2025/03/09/aslan/'
print(get_text_from_lenta_article(url, chrome_driver))


Date: 00:01, 9 марта 2025
Category: Силовые структуры
Title: 20 лет назад спецназ ликвидировал президента Ичкерии Аслана Масхадова. Как спецслужбы охотились на лидера чеченских сепаратистов?
Author: Владимир Седов
(Редактор отдела «Силовые структуры»)

=== Article Content ===

***
Аслан Масхадов родился 21 сентября 1951 года в селе Шокай Казахской ССР, куда его многодетную семью выслали в 1943 году. Масхадовы вернулись на Кавказ после реабилитации чеченских и ингушских народов лишь в 1957 году. Будущий глава ЧРИ заканчивал школу в селе Надтеречное на территории Чечено-Ингушской АССР.
После школы 18-летний Масхадов поступил в Тбилисское высшее артиллерийское командное училище, которое окончил в 1972 году и по распределению отправился на Дальний Восток. Шесть лет, с 1972 по 1978 год, он служил в Дальневосточном военном округе на берегуозера Ханка(ныне Приморский край).
К 1978 году Масхадов дослужился до заместителя командира дивизии. Затем он поступил в Ленинградскую военно-артиллерийску

In [14]:
data_test_web = data_test.copy()

In [ ]:
# get text from the pages and add it to the dataframe
for url in url_list[:5]:
    if 'expert.ru'in url:
        text = get_text_from_expert(url)
        # print(f"Text from {url}:\n{text[:300]}...\n")  # Print first 300 characters for brevity
        # add column with text
        data_test_web.loc[data_test_web['URL'] == url, 'web_text'] = text
    elif 'ria.ru' in url:
        text = get_text_from_ria(url)
        # add column with text
        data_test_web.loc[data_test_web['URL'] == url, 'web_text'] = text
    else:
        text = get_text_from_lenta(url)


data_test_web.head()

Text from https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/:
Серьезнее других от введения западных санкций пострадали отрасли, зависимые от импорта, — это нефтегаз и связанные с ним поставщики энергетического оборудования, которых отлучили от сервисных услуг и западных технологий добычи, а также машиностроение, заявил «Эксперту» директор Института народнохозяйственного прогнозирования РАН Александр Широв. Директор по экономической политике НИУ ВШЭ Юрий Симачев добавил к этому списку ряд других отраслей: базовая электроника, отдельные направления IT (где, ...

Text from https://expert.ru/mnenie/denis-manturov-gosudarstvo-v-ramkakh-natsproektov-obespechivaet-preferentsii-rossiyskim-predpriyatiya/:
Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологического суверенитета. Кого вы видите основным инвестором в них? Будут ли предоставлены частному бизнесу преференции при его готовности вкладываться в эти мегапроекты и могут ли в них уч

,URL,lib_text,web_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...",NaN
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,NaN
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...",NaN


In [50]:
data_test_web.to_csv('tmp/data_test_web.csv', index=False)